In [1]:
from BoW import getCountVector
import pandas as pd
from torch import nn
import torch
from ffnn import FFNN
import numpy as np
from scipy.sparse import coo_matrix

C:\Users\jespe\AppData\Local\Temp\ipykernel_5524\2233860815.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv("Archive/arxiv_train.csv", nrows=100)
vectoriser = getCountVector(df)

In [3]:
coo = coo_matrix(vectoriser)

values = coo.data
indices = np.vstack((coo.row, coo.col))

i = torch.LongTensor(indices)
v = torch.FloatTensor(values)
shape = coo.shape

tensor = torch.sparse.FloatTensor(i, v, torch.Size(shape)).to_dense()

C:\Users\jespe\AppData\Local\Temp\ipykernel_5524\2715430877.py:10: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:620.)
  tensor = torch.sparse.FloatTensor(i, v, torch.Size(shape)).to_dense()


In [4]:
tensor.shape

torch.Size([100, 3606])

In [6]:
model = FFNN()

output = model(tensor[0])
output

tensor([-0.1138], grad_fn=<ViewBackward0>)